In [7]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
import pandas as pd
import requests
from sentence_transformers import SentenceTransformer

In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

## Load data

Three data sources:

**CGFP** (big Excel spreadsheet) has the list of food products that we want to study
* `Product GTIN or UPC`: globally unique identifier, but unfortunately it's sparsely populated
* `Vendor`: the parent company, such as `"GENERAL MILLS REFRIGERATE"`
* `Brand Name`: a consumer-facing name for a whole series of products, such as `"YOPLAIT"`
* `Product Type`: a specific food product, maybe abbreviated, like `"YOGURT STRWBRY BANANA BASH TRIX"`
* `Level of Processing`: NOVA group, determined manually by CGFP
* does not have a list of ingredients or nutrition facts

**USDA** (branded_food.csv) can link to lists of ingredients
* `gtin_upc`: the same globally unique identifier, but matching has low recall
* `brand_owner`: the parent company, such as `"GENERAL MILLS SALES INC."`
* `brand_name`: a consumer-facing name for a whole series of products, such as `"Yoplait"`
* `subbrand_name`: not always present, but in this case `"Trix"`
* `short_description`: a specific food product, maybe abbreviated, like `"Yplt Trix StrwbrryBan LF Yogrt"`
* `ingredients`: what we want!

**OFF** (Open Food Facts) can link to lists of ingredients and independently-determined NOVA groups
* `code`: _sometimes_ this is the GIN/UPC, but often not, which makes it a low-recall way to match to CGFP, which also needs to be sanity-checked
* `product_name`: a specific food product, like `"Yoplait, trix low fat yogurt, strawberry banana bash"`
* `brands`: a consumer-facing name for a whole series of products, such as `"Yoplait USA"`
* `nova_group`: NOVA group, determined manually by OFF volunteers; a high-accuracy match to CGFP's determination
* `ingredients_text`: what we want!
* nothing says that a given food product is, for example, General Mills

A small sample of CGFP↔USDA and CGFP↔OFF ground truth matches can be made through GTIN/UPC. In the case of CGFP↔OFF, these have to be further sanity-checked by ChatGPT to exclude spurious matches from the fact that OFF's `code` is not always GTIN/UPC.

Then we want to see how much closer brand names and product descriptions are for the true matches in an embedding space than random (and usually false) pairs are. Can closeness in an embedding space, possibly with ChatGPT clean-up provide higher-recall matches?

### Load CGFP

In [5]:
cgfp = pd.read_csv(
    "~/Box/dsi-core/11th-hour/good-food-purchasing/CONFIDENTIAL_GFPP Product Attribute List_8.26.25.csv",
    dtype=str,
)[
    [
        "Product GTIN or UPC",
        "Vendor",
        "Brand Name",
        "Product Type",
        "Level of Processing",
    ]
]
cgfp_with_upc = cgfp[
    cgfp["Product GTIN or UPC"].notna() & ~cgfp["Product GTIN or UPC"].isin(["#REF!"])
].copy()
cgfp_with_upc["Product GTIN or UPC"] = cgfp_with_upc["Product GTIN or UPC"].apply(
    lambda x: f"{int(float(x.replace(' ', ''))):014d}"
)

### Load USDA

In [6]:
usda = pd.read_csv(
    "~/Box/dsi-core/11th-hour/good-food-purchasing/branded_food.csv",
    dtype=str,
)[
    [
        "gtin_upc",
        "brand_owner",
        "brand_name",
        "subbrand_name",
        "short_description",
        "ingredients",
    ]
]
usda["gtin_upc"] = (
    usda["gtin_upc"]
    .str.replace("-", "")
    .str.replace(" ", "")
    .str.replace("`", "")
    .str.replace("X", "")
    .str.replace(">", "")
    .apply(
        lambda x: {
            "NIELSENUK0002": np.nan,
            "OldCountryStore": np.nan,
            "JARLSBERG": np.nan,
            "BOOST": np.nan,
            "HAPPYKIDORGANICS": np.nan,
        }.get(x, x)
    )
    .astype(np.float64)
)
usda_with_upc = usda[usda["gtin_upc"].notna()].copy()
usda_with_upc["gtin_upc"] = usda_with_upc["gtin_upc"].apply(lambda x: f"{int(x):014d}")

### Load OFF

It's too big to load as a CSV or JSONL all the time; use their Parquet format. Since the Parquet needs to be reformatted (to pick only the first English product name and ingredients list), do the manipulation in Awkward Arrays.

In [59]:
off = ak.from_parquet(
    "~/Box/dsi-core/11th-hour/good-food-purchasing/openfoodfacts.parquet",
    columns=["code", "brands", "product_name", "nova_group", "ingredients_text"],
)
ingredients_array = ak.firsts(off["ingredients_text", "text"][off["ingredients_text", "lang"] == "en"])
product_name_array = ak.firsts(off["product_name", "text"][off["product_name", "lang"] == "en"])
off = ak.zip(
    {
        "code": off["code"],
        "brands": off["brands"],
        "product_name": product_name_array,
        "nova_group": off["nova_group"],
        "ingredients_text": ingredients_array,
    }
)

It's prohibitively expensive to convert the whole Awkward Array into Pandas, so just convert the `code` to be able to do a JOIN, then map the selected indexes into the Awkward Array.

In [64]:
off_df = pd.DataFrame({"code": ak.to_list(off["code"])}).reset_index()

## Matching

### Matching CGFP and USDA

In [66]:
cgfp_usda = cgfp_with_upc.merge(usda_with_upc, left_on="Product GTIN or UPC", right_on="gtin_upc")

In [75]:
len(cgfp_usda) / len(cgfp)

0.02688784024224474

In [72]:
cgfp_usda.take(np.random.permutation(len(cgfp_usda))[:60])[["Product Type", "brand_name", "short_description", "ingredients"]]

,Product Type,brand_name,short_description,ingredients
1223,ACT II BUTTER LOVERS POPCORN,ACT II,NaN,"Popping Corn, Palm Oil, Salt, Less than 2% of:..."
1128,SPECIAL K BAR CHOC PNUT BTR,Kellogg's,Special K Protein Bars,"INGREDIENTS: SOY PROTEIN ISOLATE, POLYDEXTROSE..."
567,"YOGURT, STRAWBERRY PARFAIT BAG (58119)",Yoplait,Yplt Strawberry PrftPro LF Yog,"Cultured Grade A Low Fat Milk, Sugar, Modified..."
870,"PIZZA 6"" ROUND CHEESE WG",Nardone Bros,6 Round Whole Wheat Cheese Pizza,"CRUST: Water, Whole Wheat Flour, Enriched Flou..."
281,BEAN BLACK LOW SODIUM,Bush's Best,Bush's Black Bn (LS) 108 oz,"Prepared Black Beans, Water, Salt, Calcium Chl..."
1772,CAMP N E CLAM CHOWDER SOUP,CAMPBELL'S,NaN,"INGREDIENTS: CLAM STOCK, POTATOES, WATER, CLAM..."
130,CHICKEN POPCORN WHL GRN FC,Tyson,TYS CN FC WG CSPY PPN CKN 32.79 LB,"Chicken, water, textured soy protein concentra..."
2115,BUN HAMBURGER WHL GRAIN SLCD,Bake Crafters,"Hamburger Buns, WG, Sliced, 3""","Whole Wheat Flour, Enriched Wheat Flour (Wheat..."
1967,SUGAR GRANULATED (786843),NaN,NaN,SUGAR
2097,APTZ CHEESE STICK BRD WHL GRAIN,FARM RICH,FR RED SODIUM CHEESE STICK 8/3#,LOW MOISTURE PART SKIM MOZZARELLA CHEESE (PAST...
